### Ceci est un mini projet personnel d'apprentissage et d'application des notions en DataViz avec (Python, kaggle Dataset, Excel, et Tableau)
Basé sur un tuto de Mo Chen => https://www.youtube.com/watch?v=nl9eZl1IOKI

In [1]:
# Décommentez le code pip install ci-dessous si vous n'avez pas encore installé ces bibliothèques
#!pip install pandas
#!pip install zipfile
#!pip install kaggle

# importer la bibliothèque pandas
import pandas as pd

# importer la bibliothèque zipfile (nous l'utiliserons pour extraire le fichier téléchargé depuis Kaggle)
import zipfile

# importer la bibliothèque kaggle (nous l'utiliserons pour télécharger le jeu de données par programme depuis Kaggle)
import kaggle

In [1]:
# télécharger le jeu de données de kaggle à l'aide de l'API Kaggle
!kaggle datasets download -d hmavrodiev/london-bike-sharing-dataset

100%|█████████████████████████████████████████| 165k/165k [00:00<00:00, 557kB/s]
100%|█████████████████████████████████████████| 165k/165k [00:00<00:00, 556kB/s]


In [3]:
# on va extraire le fichier du fichier zip téléchargé
zipfile_name = 'london-bike-sharing-dataset.zip'
with zipfile.ZipFile(zipfile_name, 'r') as file:
    file.extractall()

In [4]:
# Lecture du fichier csv en tant que dataframe pandas 
bikes = pd.read_csv("london_merged.csv")

In [5]:
# On va explorer le dataset
bikes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17414 entries, 0 to 17413
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   timestamp     17414 non-null  object 
 1   cnt           17414 non-null  int64  
 2   t1            17414 non-null  float64
 3   t2            17414 non-null  float64
 4   hum           17414 non-null  float64
 5   wind_speed    17414 non-null  float64
 6   weather_code  17414 non-null  float64
 7   is_holiday    17414 non-null  float64
 8   is_weekend    17414 non-null  float64
 9   season        17414 non-null  float64
dtypes: float64(8), int64(1), object(1)
memory usage: 1.3+ MB


In [6]:
# Dimensions de notre dataset
bikes.shape

(17414, 10)

In [7]:
# Affichons un aperçu
bikes

,timestamp,cnt,t1,t2,hum,wind_speed,weather_code,is_holiday,is_weekend,season
0,2015-01-04 00:00:00,182,3.0,2.0,93.0,6.0,3.0,0.0,1.0,3.0
1,2015-01-04 01:00:00,138,3.0,2.5,93.0,5.0,1.0,0.0,1.0,3.0
2,2015-01-04 02:00:00,134,2.5,2.5,96.5,0.0,1.0,0.0,1.0,3.0
3,2015-01-04 03:00:00,72,2.0,2.0,100.0,0.0,1.0,0.0,1.0,3.0
4,2015-01-04 04:00:00,47,2.0,0.0,93.0,6.5,1.0,0.0,1.0,3.0
...,...,...,...,...,...,...,...,...,...,...
17409,2017-01-03 19:00:00,1042,5.0,1.0,81.0,19.0,3.0,0.0,0.0,3.0
17410,2017-01-03 20:00:00,541,5.0,1.0,81.0,21.0,4.0,0.0,0.0,3.0
17411,2017-01-03 21:00:00,337,5.5,1.5,78.5,24.0,4.0,0.0,0.0,3.0
17412,2017-01-03 22:00:00,224,5.5,1.5,76.0,23.0,4.0,0.0,0.0,3.0


In [8]:
# On compte les valeurs uniques dans la colonne weather_code
bikes.weather_code.value_counts()

weather_code
1.0     6150
2.0     4034
3.0     3551
7.0     2141
4.0     1464
26.0      60
10.0      14
Name: count, dtype: int64

In [9]:
# On compte les valeurs uniques dans la colonne season
bikes.season.value_counts()

season
0.0    4394
1.0    4387
3.0    4330
2.0    4303
Name: count, dtype: int64

In [10]:
# On va spécifier les colonnes que nous allons utiliser
new_cols_dict ={
    'timestamp':'time',
    'cnt':'count', 
    't1':'temp_real_C',
    't2':'temp_feels_like_C',
    'hum':'humidity_percent',
    'wind_speed':'wind_speed_kph',
    'weather_code':'weather',
    'is_holiday':'is_holiday',
    'is_weekend':'is_weekend',
    'season':'season'
}

# Et renommer ces colonnes
bikes.rename(new_cols_dict, axis=1, inplace=True)

In [11]:
# On va changer les valeurs d'humidité en pourcentage (c'est-à-dire une valeur entre 0 et 1)
bikes.humidity_percent = bikes.humidity_percent / 100

In [12]:
# On va créer un dictionnaire des saisons afin de pouvoir faire correspondre les nombres entiers 0-3 aux valeurs écrites
season_dict = {
    '0.0':'spring',
    '1.0':'summer',
    '2.0':'autumn',
    '3.0':'winter'
}

# On va créer un dictionnaire météo afin de pouvoir faire correspondre les nombres entiers aux valeurs écrites réelles
weather_dict = {
    '1.0':'Clear',
    '2.0':'Scattered clouds',
    '3.0':'Broken clouds',
    '4.0':'Cloudy',
    '7.0':'Rain',
    '10.0':'Rain with thunderstorm',
    '26.0':'Snowfall'
}

# changer le type de données de la colonne "saisons" en chaîne de caractères
bikes.season = bikes.season.astype('str')
# en faisant correspondre les valeurs 0 à 3 aux saisons réellement écrites
bikes.season = bikes.season.map(season_dict)

# changement du type de données de la colonne météo en chaîne
bikes.weather = bikes.weather.astype('str')
# mise en correspondance des valeurs avec les conditions météorologiques écrites
bikes.weather = bikes.weather.map(weather_dict)

In [13]:
# vérification de notre Dataset pour voir si les mappages ont fonctionné
bikes.head()

,time,count,temp_real_C,temp_feels_like_C,humidity_percent,wind_speed_kph,weather,is_holiday,is_weekend,season
0,2015-01-04 00:00:00,182,3.0,2.0,0.930,6.0,Broken clouds,0.0,1.0,winter
1,2015-01-04 01:00:00,138,3.0,2.5,0.930,5.0,Clear,0.0,1.0,winter
2,2015-01-04 02:00:00,134,2.5,2.5,0.965,0.0,Clear,0.0,1.0,winter
3,2015-01-04 03:00:00,72,2.0,2.0,1.000,0.0,Clear,0.0,1.0,winter
4,2015-01-04 04:00:00,47,2.0,0.0,0.930,6.5,Clear,0.0,1.0,winter


In [ ]:
# Exporter notre dataset final dans un fichier Excel que nous utiliserons dans nos visualisations Tableau. 
# Le fichier sera 'london_bikes_final.xlsx' et le nom de la feuille sera 'Data'
bikes.to_excel('london_bikes_final.xlsx', sheet_name='Data')